In [152]:
import wandb

api = wandb.Api()
runs = api.runs(
    "chiyuzh/cyberbench",
    filters={
        "state": "finished",
        # "config.model": {"$in": [
        #     "gpt-4o",
        #     "phi-3-medium-128k-instruct-1",
        #     "microsoft/Phi-3.5-mini-instruct",
        #     "meta-llama-3-70b-instruct-4",
        #     "microsoft/Phi-3-mini-128k-instruct",
        #     "random-agent",
        # ]}
    }
)

#print(run.history())
print(len(runs))


70


In [153]:
print(runs[0].config)
print(runs[0].summary)

{'LLM': 'random', 'env': 'PhishingTextFewShot'}
{'_runtime': 11.00635266304016, '_step': 739, '_timestamp': 1723148138.2560518, '_wandb': {'runtime': 11}, 'episode/normalized_score': 0.54, 'episode/rollout': {'_latest_artifact_path': 'wandb-client-artifact://d7zwitsbg90ipljua1les49d1emon95gotdkxrbqeqa1p0spqsechodyaganx5e8hxqdv61ccv2w31kdral3ebtcq5pdja0v08fj8cinfibd0yk593bjep06lvfwdhvw:latest/episode/rollout.table.json', '_type': 'table-file', 'artifact_path': 'wandb-client-artifact://0tfthpj7s3ynyg5twjqtz317gx3gfx4zyl9r7uuilrijimm7suvn06pn8el1qq5hix4tihyjn6hcgl0qfst4gmllm977wi4puj5rs30g63a5jytktdagudz1i74g2wo2/episode/rollout.table.json', 'ncols': 11, 'nrows': 740, 'path': 'media/table/episode/rollout_739_b4f3e49e45104f8e7844.table.json', 'sha256': 'b4f3e49e45104f8e78447b62e4cf6b7108bd24bcd70d892b9681f24fa9b47054', 'size': 11387828}, 'episode/score': 270, 'final/normalized_score': 0.54, 'metrics/acc': 0.54, 'metrics/macf1': 0.5279146141215106, 'metrics/macpre': 0.5319388214125056, 'met

In [154]:
# Conver Wandb RUNS to a pandas dataframe
import pandas as pd
import numpy as np

runs_data = []
for run in runs:
    run_data = dict(run.summary)
    run_data["model"] = run.config["LLM"]
    run_data["env"] = run.config["env"]
    runs_data.append(run_data)

runs_df = pd.DataFrame(runs_data)

# Keep env, model, Max Normalized Score
runs_df = runs_df[["env", "model", "final/normalized_score"]]
runs_df = runs_df.sort_values(by="env")

runs_df


,env,model,final/normalized_score
69,CVEFix,phi-3-medium-h100,0.346667
67,CVEFix,llama-31-h100,0.717015
66,CVEFix,gpt-4o-gs,0.710786
65,CVEFix,random,0.847910
68,CVEFix,phi-35-h100,0.608603
...,...,...,...
42,PhishingWebFewShot,phi-3,0.686000
2,PhishingWebFewShot,gpt-4o-gs,0.928000
1,PhishingWebFewShot,random,0.534000
55,PhishingWebFewShot,phi-35-h100,0.676000


In [155]:
set(runs_df.model.values.tolist())

{'gpt-4o-gs',
 'llama-31-h100',
 'meta-llama-3-1-70b-instruct-1',
 'phi-3',
 'phi-3-medium-h100',
 'phi-35-h100',
 'random'}

In [156]:
aug_task = ["CodeVulnerabilityDetectionFewShot", "CyberThreatIntelligenceMultiChoiceQuestionsWithContext",
            "PhishingTextFewShot", "PhishingWebFewShot"]

aug_run_df = runs_df[runs_df["env"].isin(aug_task)]

env_name_standard = {"CodeVulnerabilityDetectionFewShot": "4-shot Vulner. Detec.", 
                     "CyberThreatIntelligenceMultiChoiceQuestionsWithContext": "CTI-MCQA w Context",
                    "PhishingTextFewShot": "4-Shot Malic. Text Detec.",
                    "PhishingWebFewShot": "4-Shot Malic. Web Detec."}

aug_run_df["env"] = aug_run_df["env"].map(env_name_standard)

/var/folders/2l/tg3jz6ps50j68v4nmxs57dnc0000gn/T/ipykernel_1217/208271861.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [157]:
# filter rows:
# main results
main_tasks = ["PhishingText", "PhishingWeb", "CyberThreatIntelligenceMultiChoiceQuestions", 
              "CodeVulnerabilityDetection", "CVEFix", "CyberBattleChain10"]

main_run_df = runs_df[runs_df["env"].isin(main_tasks)]

In [158]:
env_name_standard = {"CodeVulnerabilityDetection": "Vulner. Detec.", "CyberBattleChain10": "Net. Intrus.",
                 "CyberThreatIntelligenceMultiChoiceQuestions": "CTI-MCQA", "PhishingText": "Malic. Text Detec.",
                 "PhishingWeb": "Malic. Web Detec.", "CVEFix": "CVEFix"}
main_run_df["env"] = main_run_df["env"].map(env_name_standard)


/var/folders/2l/tg3jz6ps50j68v4nmxs57dnc0000gn/T/ipykernel_1217/1157513022.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [159]:
model_name_standard = {'gpt-4o-gs': "GPT-4o", 'llama-31-h100': "LLaMA-3.1", 'meta-llama-3-1-70b-instruct-1': "LLaMA-3.1",
 'phi-3': "Phi-3", 'phi-3-medium-h100': "Phi-3 Med.", 'phi-35-h100': "Phi-3.5", 'random': "Baseline"}

main_run_df["model"] = main_run_df["model"].map(model_name_standard)

/var/folders/2l/tg3jz6ps50j68v4nmxs57dnc0000gn/T/ipykernel_1217/1829732603.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go


fig = px.line_polar(main_run_df,
                    r="final/normalized_score",
                    theta="env",
                    line_group="model",
                    color="model",
                    line_close=True,
                    #template="plotly_dark",
                    title="Normalized score per task",)
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 1])))

# Move legend to the bottom left
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.03,
    xanchor="right",
    x=0.93
))
# Hide title
fig.update_layout(title="")
fig.update_polars(bgcolor='white', angularaxis_linecolor = "black", radialaxis_gridcolor = "#d6d6d6", 
                  angularaxis_gridcolor = "#d6d6d6", radialaxis_linecolor = "#d6d6d6")

# Hide legen title
fig.update_layout(legend_title="")
# Change figure size to be
fig.write_image("radar_chart.png", width=800, height=800)
fig.show()


In [161]:
aug_run_df["model"] = aug_run_df["model"].map(model_name_standard)

/var/folders/2l/tg3jz6ps50j68v4nmxs57dnc0000gn/T/ipykernel_1217/4253450668.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [162]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go


fig = px.line_polar(aug_run_df,
                    r="final/normalized_score",
                    theta="env",
                    line_group="model",
                    color="model",
                    line_close=True,
                    #template="plotly_dark",
                    title="Normalized score per task",)
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 1])))

# Move legend to the bottom left
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.03,
    xanchor="right",
    x=0.86
))
# Hide title
fig.update_layout(title="")
fig.update_polars(bgcolor='white', angularaxis_linecolor = "black", radialaxis_gridcolor = "#d6d6d6", 
                  angularaxis_gridcolor = "#d6d6d6", radialaxis_linecolor = "#d6d6d6")

# Hide legen title
fig.update_layout(legend_title="")
# Change figure size to be
fig.write_image("radar_chart_aug.png", width=800, height=800)
fig.show()


In [137]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go

# Sort runs_df theta according to gpt-4o best max normalized score
runs_df = runs_df.sort_values(by=["env", "final/normalized_score"], ascending=True)

fig = px.line_polar(runs_df,
                    r="final/normalized_score",
                    theta="env",
                    line_group="model",
                    color="model",
                    line_close=True, template="plotly_dark", title="Normalized Score per task")
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 100])))

fig.show()

In [6]:
# Create a group bar chart with the average score per game for each model
import plotly.express as px
import plotly.graph_objects as go

df = runs_df.copy()

# sort by max score for gpt-4o
#df = df.sort_values(by=["model", "Max Normalized Score"], ascending=True)
# Strip the extension from game name
df["game"] = df["game"].map(lambda x: x.split(".")[0])

# Filterout game lurking and hollywood
#df = df[~df["game"].isin(["lurking", "hollywood"])]

# Add new column to order difficulty
df["order"] = df["Difficulty"].map({"Possible": 2, "Difficult": 1, "Extreme": 0})

df = df.sort_values(by=["order", "model", "Max Normalized Score"], ascending=True)

# Add new column color for each game based on the game difficulty
def color(game, diff):
    if diff == "Possible":
        return f"<span style='color:green'>{game}</span>"
    elif diff == "Difficult":
        return f"<span style='color:orange'>{game}</span>"
    elif diff == "Extreme":
        return f"<span style='color:red'>{game}</span>"
    else:
        raise ValueError("Unknown difficulty")

df["color"] = df.apply(lambda x: color(x["game"], x["Difficulty"]), axis=1)


fig = px.bar(df, x="Max Normalized Score", y="game", color="model", barmode="group", title="Max Normalized Score per game", height=1000)

# Sort the y axis by game difficulty and then by max score for gpt-4o
#fig.update_yaxes(categoryorder="array", categoryarray=df.sort_values(by=["order", "model", "Max Normalized Score"], ascending=True))
fig.update_yaxes(tickvals=df["game"], tickmode="array", ticktext=df["color"])

# Make x axis range from 0 to 100
fig.update_xaxes(range=[0, 100])

# sort game title by max score for gpt-4o
#fig.update_yaxes(categoryorder="total ascending")


KeyError: 'game'